In [ ]:
# Cell 1: Setup and Configuration
import requests
import json
import os
from datetime import datetime
from dotenv import load_dotenv

# Load environment variables from a .env file
load_dotenv()

# --- CONFIGURATION ---
API_KEY = os.getenv("LINEAR_API_KEY")

# --- API Setup ---
API_URL = "https://api.linear.app/graphql"
HEADERS = {
    "Authorization": API_KEY, # No "Bearer" prefix
    "Content-Type": "application/json"
}

# Helper function to run GraphQL queries/mutations
def run_linear_query(query, variables=None):
    """A simple function to post a query to the Linear API."""
    if not API_KEY:
        print("❌ ERROR: LINEAR_API_KEY not found in environment.")
        return None # Return None if key is missing

    payload = {'query': query}
    if variables:
        payload['variables'] = variables

    try:
        response = requests.post(API_URL, headers=HEADERS, json=payload)
        response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)

        json_response = response.json()
        if "errors" in json_response:
            print(f"❌ GraphQL Error: {json_response['errors']}")
            # Consider logging the error instead of just printing in a real app
            # For simplicity, returning None on GraphQL error
            return None
        return json_response # Return the JSON data on success

    except requests.exceptions.HTTPError as e:
        print(f"❌ HTTP Error: {e.status_code} - {e.response.text}")
    except requests.exceptions.RequestException as e:
        print(f"❌ Request Error: {e}")
    except json.JSONDecodeError:
        print(f"❌ JSON Decode Error: Could not parse response: {response.text}")

    return None # Return None on any exception

if not API_KEY:
    print("❌ ERROR: LINEAR_API_KEY not found in environment variables.")
else:
    print("✅ Setup complete. API key loaded. Ready to proceed.")

In [ ]:
# Cell 2: Function - Fetch All Team Assets (Left Panel)

def fetch_all_team_assets(team_key: str):
    """
    Fetches all projects and issues for a given team.
    This data will populate the left-side panel of the app.
    """
    print(f"🔎 Fetching all assets for team '{team_key}'...")
    # Get team ID first
    get_team_id_query = "query TeamByKey($key: String!) { teams(first: 1, filter: { key: { eq: $key } }) { nodes { id name key } } }"
    team_data_response = run_linear_query(get_team_id_query, {"key": team_key})
    if not (team_data_response and team_data_response.get("data", {}).get("teams", {}).get("nodes")):
         print(f"❌ Could not find team with key '{team_key}'.")
         return None
    team_id = team_data_response["data"]["teams"]["nodes"][0]["id"]
    print(f"✅ Found Team ID: {team_id}")


    query = """
    query TeamAssets($teamId: String!) {
      team(id: $teamId) {
        id
        name

        projects(first: 200) {
          nodes {
            id
            name
            state # Project state is just a string
          }
        }

        issues(first: 200) {
          nodes {
            id
            title
            identifier
            state { id, name } # Issue state is an object
            priority
            labels { nodes { id, name } } # Fetch existing labels too
          }
        }
      }
    }
    """
    variables = {"teamId": team_id}
    data = run_linear_query(query, variables)

    if data and data.get('data') and data['data'].get('team'):
        print("✅ Successfully fetched assets.")
        return data['data']['team']
    else:
        print(f"❌ Could not fetch assets for team ID '{team_id}'.")
        return None

In [ ]:
# Cell 3: Function - Fetch Workspace Properties (Right Panel)

def fetch_workspace_properties(team_key: str):
    """
    Fetches all properties needed to populate the dropdowns
    in the app's right-side editing panel.
    """
    print(f"🔎 Fetching all workspace properties...")
    # Get team ID first
    get_team_id_query = "query TeamByKey($key: String!) { teams(first: 1, filter: { key: { eq: $key } }) { nodes { id name key } } }"
    team_data_response = run_linear_query(get_team_id_query, {"key": team_key})
    if not (team_data_response and team_data_response.get("data", {}).get("teams", {}).get("nodes")):
         print(f"❌ Could not find team with key '{team_key}' for property fetching.")
         return None
    team_id = team_data_response["data"]["teams"]["nodes"][0]["id"]

    query = """
    query TeamProps($teamId: String!) {
      # 1. Get properties relevant to the specific team (Issue Labels/Statuses)
      team(id: $teamId) {
        labels { nodes { id, name, color } } # Issue Labels
        states { nodes { id, name, type } } # Issue Statuses
      }

      # 2. Get properties relevant to the whole organization
      organization {
        projectStatuses { id, name } # Project Statuses (List, no 'nodes')
      }

      # 3. Get Project Labels (Separate from Issue Labels)
      projectLabels {
         nodes { id, name, color }
      }

      # 4. Get all projects (for the "Move issue to project" dropdown)
      projects(first: 200) {
        nodes { id, name }
      }
    }
    """
    variables = {"teamId": team_id} # Pass teamId for context
    data = run_linear_query(query, variables)

    if data and data.get('data'):
        print("✅ Successfully fetched properties.")
        # Organize the data cleanly
        try:
            properties = {
                "issueLabels": data['data']['team']['labels']['nodes'],
                "issueStatuses": data['data']['team']['states']['nodes'],
                "projectStatuses": data['data']['organization']['projectStatuses'],
                # --- ADDED Project Labels ---
                "projectLabels": data['data']['projectLabels']['nodes'],
                # --------------------------
                "allProjects": data['data']['projects']['nodes'],
                "issuePriorities": [ # Hardcoded priorities (0-4)
                    {"name": "No priority", "value": 0},
                    {"name": "Urgent", "value": 1},
                    {"name": "High", "value": 2},
                    {"name": "Medium", "value": 3},
                    {"name": "Low", "value": 4}
                ]
            }
            return properties
        except (KeyError, TypeError) as e:
            print(f"❌ Error organizing properties data: {e}. Raw data: {data.get('data')}")
            return None
    else:
        print("❌ Could not fetch workspace properties.")
        return None

In [ ]:
# Cell 4: Function - Bulk Update Projects

def bulk_update_projects(project_ids: list, updates: dict):
    """
    Updates a list of projects with the provided properties.
    Returns a list of results (success/failure for each ID).
    """
    print(f"\n🚀 Starting bulk update for {len(project_ids)} project(s)...")
    results = []
    input_payload = updates.copy()

    # Normalize state to Linear enum (lowercase)
    if "state" in input_payload and input_payload["state"]:
        state_value = str(input_payload["state"]).lower()
        allowed = {"backlog", "planned", "started", "paused", "canceled", "completed"}
        if state_value == "in progress": state_value = "started" # Handle common name variation
        if state_value in allowed:
            input_payload["state"] = state_value
        else:
             print(f"⚠️ Invalid project state: '{updates['state']}'. Removing from update.")
             input_payload.pop("state", None) # Use pop with None default

    # Dates -> ISO timestamp format (YYYY-MM-DDTHH:MM:SS.fffZ)
    for k in ["startDate", "targetDate"]:
        if k in input_payload and input_payload[k]:
            v = input_payload[k]
            if len(v) == 10 and v.count("-") == 2: # Check if it's YYYY-MM-DD
                try:
                    input_payload[k] = v + "T00:00:00.000Z" # Append time for API
                except ValueError:
                    print(f"⚠️ Invalid {k} format: {v}. Removing from update.")
                    input_payload.pop(k, None)

    # Ensure labelIds is always a list if present and not None
    if "labelIds" in input_payload and input_payload["labelIds"] is not None:
        if not isinstance(input_payload["labelIds"], list):
            input_payload["labelIds"] = [input_payload["labelIds"]]
            print(f"⚠️ Converted single project labelId to list.")
        # Add validation here if you fetch specific project label IDs later


    mutation = """
    mutation ProjectUpdate($id: String!, $input: ProjectUpdateInput!) {
      projectUpdate(id: $id, input: $input) {
        success
        project { id, name, state }
      }
    }
    """
    for pid in project_ids:
        try:
            if not input_payload:
                 print(f"    ⚠️ Skipping project {pid}: No valid updates provided.")
                 results.append({"id": pid, "success": False, "error": "No valid updates"})
                 continue

            print(f"  - Updating project {pid} with payload: {input_payload}") # Debug print
            data = run_linear_query(mutation, {"id": pid, "input": input_payload})
            ok = data.get("data", {}).get("projectUpdate", {}).get("success", False) if data else False
            if ok:
                project_name = data["data"]["projectUpdate"]["project"]["name"]
                results.append({"id": pid, "success": True, "name": project_name})
                print(f"    ✅ Success: '{project_name}' updated.")
            else:
                error_msg = data.get("errors", "API reported failure") if data else "API reported failure"
                results.append({"id": pid, "success": False, "error": error_msg})
                print(f"    ❌ Failed to update {pid} ({error_msg}).")
        except Exception as e:
            print(f"    ❌ Failed to update project {pid}: {e}")
            results.append({"id": pid, "success": False, "error": str(e)})

    print("✨ Bulk project update process complete.")
    return results

In [ ]:
# Cell 5: Function - Bulk Update Issues

def bulk_update_issues(issue_ids: list, updates: dict):
    """
    Updates a list of issues with the provided properties.
    Returns a list of results (success/failure for each ID).
    """
    print(f"\n🚀 Starting bulk update for {len(issue_ids)} issue(s)...")
    results = []
    input_payload = updates.copy()

    # Ensure labelIds is always a list if present
    if "labelIds" in input_payload and input_payload["labelIds"] is not None and not isinstance(input_payload["labelIds"], list):
        input_payload["labelIds"] = [input_payload["labelIds"]]

    # Ensure priority is integer if present
    if "priority" in input_payload and input_payload["priority"] is not None:
         try:
             input_payload["priority"] = int(input_payload["priority"])
         except (ValueError, TypeError):
             print(f"⚠️ Invalid priority value: {input_payload['priority']}. Removing from update.")
             input_payload.pop("priority", None)


    mutation = """
    mutation IssueUpdate($id: String!, $input: IssueUpdateInput!) {
      issueUpdate(id: $id, input: $input) {
        success
        issue { id identifier state { name } }
      }
    }
    """
    for iid in issue_ids:
        try:
            if not input_payload:
                 print(f"    ⚠️ Skipping issue {iid}: No valid updates provided.")
                 results.append({"id": iid, "success": False, "error": "No valid updates"})
                 continue

            print(f"  - Updating issue {iid} with payload: {input_payload}") # Debug print
            data = run_linear_query(mutation, {"id": iid, "input": input_payload})
            ok = data.get("data", {}).get("issueUpdate", {}).get("success", False) if data else False
            if ok:
                identifier = data["data"]["issueUpdate"]["issue"]["identifier"]
                results.append({"id": iid, "success": True, "identifier": identifier})
                print(f"    ✅ Success: '{identifier}' updated.")
            else:
                error_msg = data.get("errors", "API reported failure") if data else "API reported failure"
                results.append({"id": iid, "success": False, "error": error_msg})
                print(f"    ❌ Failed to update {iid} ({error_msg}).")
        except Exception as e:
            print(f"    ❌ Failed to update issue {iid}: {e}")
            results.append({"id": iid, "success": False, "error": str(e)})

    print("✨ Bulk issue update process complete.")
    return results

In [ ]:
# Cell 6: Test - Setup (Fetch All Data)

# ⚠️ Set your team key here
TEAM_KEY = "LIN" # Or "TEA1" or whichever key you want to test with

if API_KEY:
    # 1. Fetch all projects and issues
    team_assets = fetch_all_team_assets(TEAM_KEY)

    # 2. Fetch all available properties (including Project Labels)
    workspace_properties = fetch_workspace_properties(TEAM_KEY)

    if team_assets and workspace_properties:
        print("\n--- ✅ SETUP COMPLETE ---")

        # --- Print sample data ---
        print("\nSample Projects:")
        for p in team_assets['projects']['nodes'][:3]: # Print first 3
            print(f"  - Name: {p['name']}, ID: {p['id']}")

        print("\nSample Issues:")
        for i in team_assets['issues']['nodes'][:3]: # Print first 3
            print(f"  - Name: {i['title']}, ID: {i['id']}")

        print("\nSample Issue Labels:")
        for l in workspace_properties['issueLabels'][:3]:
            print(f"  - Name: {l['name']}, ID: {l['id']}")

        # --- ADDED: Print Project Labels ---
        print("\nSample Project Labels:")
        if not workspace_properties['projectLabels']:
             print("  (No Project Labels found in this workspace)")
        else:
            for l in workspace_properties['projectLabels'][:3]:
                print(f"  - Name: {l['name']}, ID: {l['id']}")
        # -----------------------------------

        print("\nSample Issue Statuses (for 'stateId'):")
        for s in workspace_properties['issueStatuses'][:3]:
            print(f"  - Name: {s['name']}, ID: {s['id']}")

        print("\nSample Project Statuses (for 'state'):")
        for s in workspace_properties['projectStatuses']:
            print(f"  - Name: {s['name']}") # Use Name, backend converts to lowercase type
else:
    print("🚫 API Key not loaded. Please fix Cell 1.")

In [ ]:
# Cell 7: Test - Run Bulk Updates

if 'team_assets' in locals() and 'workspace_properties' in locals():

    # --- 1. TEST BULK PROJECT UPDATE (including labels) ---
    # ⚠️ PASTE YOUR PROJECT IDs HERE (Using IDs from your Cell 6 output)
    project_ids_to_update = [
        "283bb14d-cf61-43ee-a582-c23aa9b473a7", # HydraTrack
        "cbb925bc-c298-49ad-bee0-574a63b5e65b"  # Hydrate Tracker
    ]

    # ⚠️ PASTE PROJECT LABEL IDs HERE (Using "Api" ID from your Cell 6 output)
    project_label_ids_to_apply = [
        "3a7b1dd8-7de2-4860-be1e-b5a33e640943", # Project Label: "Api"
    ]

    # ⚠️ Define the changes you want to make
    project_changes = {
        "state": "planned",  # Use lowercase type name
        "labelIds": project_label_ids_to_apply # Use the specific Project Label IDs
    }

    # Uncomment the line below to run the project update
    print("--- STARTING PROJECT UPDATE ---")
    bulk_update_projects(project_ids_to_update, project_changes)


    # --- 2. TEST BULK ISSUE UPDATE ---
    # ⚠️ PASTE YOUR ISSUE IDs HERE (Using IDs from your Cell 6 output)
    issue_ids_to_update = [
        "2a0c5bc5-6f70-4cef-be23-09bde52de296", # Create Discord Bot 2
        "7c283404-4307-4e2a-81cc-ffeeaceaf265", # Create Discord Bot
    ]

    # ⚠️ Get an Issue Label ID and Issue State ID from Cell 6 output
    test_issue_label_id = "a0eb1457-123e-4e78-ac84-662843844714" # Issue Label: "Bug"
    test_issue_state_id = "e7ab3f79-df00-4436-a160-1d4494fa47d8" # Issue Status: "In Progress"

    # ⚠️ Define the changes
    issue_changes = {
        "stateId": test_issue_state_id,
        "priority": 3, # 3 = Medium
        "labelIds": [test_issue_label_id]
    }

    # Uncomment the line below to run the issue update
    # print("\n--- STARTING ISSUE UPDATE ---")
    # bulk_update_issues(issue_ids_to_update, issue_changes)

    print("\n✅ Test cell is ready with IDs from your output.")
    print("👉 To run the test, uncomment the 'bulk_update_projects' and/or 'bulk_update_issues' lines and run this cell again.")

else:
    print("🚫 Data not loaded. Please run Cell 6 first.")